In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2024-07-15 22:32:42.678206: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 22:32:42.680507: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 22:32:42.727069: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 22:32:42.728124: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 22:32:43.508260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(
    dataset,
    timesteps,
    split_by_cement_type=False,
    train_columns=[],
    test_columns=[],
):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=train_columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=test_columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=train_columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=test_columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return model, score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

In [9]:
def evaluate_model(model, dataset):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [10]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [11]:
class BidirectionalLSTM1(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    units=128, activation="relu", return_sequences=True
                )
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="relu")
            )
        )
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [12]:
class BidirectionalLSTM2(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    units=128, activation="relu", return_sequences=True
                )
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="relu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=32, activation="relu")
            )
        )

        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [13]:
class BidirectionalLSTM3(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=128, activation="elu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="elu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=32, activation="elu")
            )
        )

        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [14]:
class BidirectionalLSTM4(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    units=128, activation="relu", return_sequences=True
                )
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="relu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=32, activation="relu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=16, activation="relu")
            )
        )

        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [15]:
class BidirectionalLSTM5(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=128, activation="elu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="elu")
            )
        )
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [16]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [17]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [18]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [19]:
def set_global_determinism():
    set_seeds()

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [20]:
index_to_save = 2
model_index = 1

<IPython.core.display.Javascript object>

In [21]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = [
    "Cement_Type_CEM B",
    "Cement_Type_CEM C",
    "Cement_Type_CP II-F-32",
    "Cement_Type_CP II-F-40",
    "Cement_Type_CP V-ARI",
    "Cement_Type_Type I-II",
    "Cement_Type_Type III",
    "Cement_Type_Type IL",
]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [22]:
results_to_save = []

results_dict = {
    "Category": "Global Model",
    "Company": "INN",
    "Plant": "INN",
    "Features": "Chemical + Mineralogical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "BidirectionalLSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [23]:
df = pd.read_csv("../../../../../../../data/processed/inn/global_dataset_inn.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [24]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        # "Cement_Type",
        # "Factory_Plant",
        "Blaine",
        # "#200",
        "#325",
        # "Final setting time",
        "Initial setting time",
        "CS1",
        "CS3",
        "CS7",
        # "Factory_Plant",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [25]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
x = df_copy.drop(["Date", "CS28", "Factory_Plant"] + CEMENT_TYPES, axis=1)
# x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
y = df_copy["CS28"]
train_columns = x.columns

<IPython.core.display.Javascript object>

In [26]:
df_copy["Factory_Plant"].value_counts()

Factory_Plant
partner_i    1223
partner_ii    1051
partner_iv          906
Name: count, dtype: int64

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [27]:
def prepare_dataset(
    dataframe_original=df,
    dataframe_copy=df_copy,
    train_size=0.8,
    test_size=0.2,
    ignore_test_set=False,
    timesteps=1,
    split_by_cement_type=True,
):
    dataframe_original = df.copy()
    dataframe_copy = df_copy.copy()
    dataframe_copy[CEMENT_TYPES] = dataframe_copy[CEMENT_TYPES].astype(int).copy()
    dates = dataframe_original["Date"].copy()
    x = dataframe_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = dataframe_copy["CS28"]
    cement_types = dataframe_copy[CEMENT_TYPES].copy()

    # Split the dataframe by plant
    plants = dataframe_original["Factory_Plant"].unique()
    train_indexes = []
    test_indexes = []
    plants_test_indexes = {}

    for plant in plants:
        plant_df = dataframe_original[dataframe_original["Factory_Plant"] == plant]
        plant_df = plant_df.sort_values(by="Date")
        plant_indices = plant_df.index
        train_end_index = int(len(plant_indices) * train_size)
        # plants_indexes[]

        if not ignore_test_set:
            train_indexes.extend(plant_indices[:train_end_index])
            test_indexes.extend(plant_indices[train_end_index:])
            plants_test_indexes[plant] = plant_indices[train_end_index:]

        else:
            train_indexes.extend(plant_indices[:train_end_index])
            test_indexes.extend(plant_indices[train_end_index:])
            plants_test_indexes[plant] = plant_indices[train_end_index:]

    train_index = dataframe_original.loc[train_indexes].sort_values(by="Date").index
    test_index = dataframe_original.loc[test_indexes].sort_values(by="Date").index

    plant_id_series = x["Factory_Plant"].copy()
    x = x.drop(["Factory_Plant"], axis=1)

    dataset = {
        "dates_train": dates[train_index].reset_index(drop=True).copy(),
        "cement_types_train": cement_types.loc[train_index]
        .reset_index(drop=True)
        .copy(),
        "x_train": x.loc[train_index].reset_index(drop=True).copy(),
        "y_train": y[train_index].reset_index(drop=True).copy(),
        "dates_test": dates[test_index].reset_index(drop=True).copy(),
        "cement_types_test": cement_types.loc[test_index].reset_index(drop=True).copy(),
        "x_test": x.loc[test_index].reset_index(drop=True).copy(),
        "y_test": y[test_index].reset_index(drop=True).copy(),
    }

    # Preprocess the dataset
    dataset = preprocess_data(dataset, None, SimpleImputer, {"strategy": "median"})
    # return dataset, plant_id_series.loc[test_index].reset_index(drop=True).copy()
    dataset["x_test"] = np.concatenate(
        [
            dataset["x_test"],
            plant_id_series.loc[test_index]
            .reset_index(drop=True)
            .copy()
            .values.reshape(-1, 1),
        ],
        axis=1,
    )
    index_list = x.columns.tolist()
    index_list.append("Factory_Plant")
    test_columns = pd.Index(index_list)

    # generate sequences (3D format)
    dataset = generate_sequences(
        dataset,
        timesteps=timesteps,
        split_by_cement_type=split_by_cement_type,
        train_columns=train_columns,
        test_columns=test_columns,
    )

    return dataset

<IPython.core.display.Javascript object>

In [28]:
def prepare_dataset_helper(timesteps):
    dataset = prepare_dataset(timesteps=timesteps)
    # Remove the last column (plant identification) for overall evaluation
    x_test_overall = np.delete(dataset["x_test"], -1, axis=2)

    # Initialize dictionaries to hold plant-specific datasets
    plant_specific_x_test = {}
    plant_specific_y_test = {}

    # Get unique plant identifiers
    unique_plants = np.unique(dataset["x_test"][:, 0, -1])

    for plant in unique_plants:
        # Create a mask for the current plant
        plant_mask = dataset["x_test"][:, 0, -1] == plant
        # Filter x_test and y_test using the mask
        x_test_plant = dataset["x_test"][plant_mask]
        y_test_plant = dataset["y_test"][plant_mask]

        # Remove the last column (plant identification) from x_test_plant
        x_test_plant = np.delete(x_test_plant, -1, axis=2)

        # Store the filtered arrays in the dictionaries
        plant_specific_x_test[plant] = x_test_plant
        plant_specific_y_test[plant] = y_test_plant

    # Output the shapes to verify
    # print("Overall x_test shape:", x_test_overall.shape)
    # for plant in plant_specific_x_test:
    #    print(f"Plant {plant} x_test shape:", plant_specific_x_test[plant].shape)
    #    print(f"Plant {plant} y_test shape:", plant_specific_y_test[plant].shape)

    plant_specific = {"x_test": plant_specific_x_test, "y_test": plant_specific_y_test}
    x_test_overall = x_test_overall.astype(float)
    dataset["x_test"] = x_test_overall
    return dataset, plant_specific

<IPython.core.display.Javascript object>

In [29]:
def print_scores_2(scores, METRICS, METRICS_DICT):
    for phase in ["test"]:
        print("******")
        print(f"[{phase.upper()}]")
        print("******")
        for metric in METRICS:
            name = METRICS_DICT[metric]
            print(
                f"{name}: %.3f (%.3f)"
                % (
                    np.mean(scores[f"{phase}_" + metric]),
                    np.std(scores[f"{phase}_" + metric]),
                )
            )
        print("\n======================\n")

<IPython.core.display.Javascript object>

In [30]:
def append_results(scores_dict):
    for plant, scores in scores_dict.items():
        results_dict_copy = results_dict.copy()
        results_dict_copy["Plant"] = plant
        results_dict_copy["Timesteps"] = timesteps
        results_dict_copy["Cross Validation"] = "Out of time"
        results_dict_copy[
            "Cross Validation Params"
        ] = '{"train_size": 0.8, "test_size": 0.2}'
        results_dict_copy["Data Shape"] = x.shape
        results_dict_copy["Model"] = f"Bi-LSTM{model_index}"
        scores = {key: [value] for key, value in scores.items()}
        df_results = fill_results_dict(results_dict_copy, scores)
        results_to_save.append(df_results)


<IPython.core.display.Javascript object>

In [31]:
def print_results_and_get_scores_dict(scores, plant_specific):
    print("=======================")
    print("Overall Score")
    print("=======================\n")

    print_scores(scores, METRICS, METRICS_DICT)

    print("=======================")
    print("Scores per plant")
    print("=======================\n")

    scores_dict = {"INN": scores}

    for plant in df_copy["Factory_Plant"].unique():
        x_test = plant_specific["x_test"][plant].astype(float)
        y_test = plant_specific["y_test"][plant].astype(float)
        dataset["x_test"] = x_test
        dataset["y_test"] = y_test
        print("=======================")
        print(
            "Plant",
            plant,
        )
        print("=======================\n")
        scores_plant = evaluate_model(model, dataset)
        scores_dict[plant] = scores_plant
        print_scores_2(scores_plant, METRICS, METRICS_DICT)
    return scores_dict

<IPython.core.display.Javascript object>

In [32]:
def get_bilstm_params(
    timesteps=1,
    activation="relu",
    callbacks=None,
    validation_split=0.0,
    verbose=0,
):
    params = {}
    params["verbose"] = verbose
    params["callbacks"] = callbacks
    params["validation_split"] = validation_split

    return params

<IPython.core.display.Javascript object>

In [33]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# Bidirectional LSTM

### BiLSTM 1

1. TIMESTEPS: 1

In [34]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM1, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

2024-07-15 22:32:49.113178: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-15 22:32:49.113220: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2024-07-15 22:32:49.113228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2024-07-15 22:32:49.113423: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.183.1
2024-07-15 22:32:49.113451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.183.1
2024-07-15 22:32:49.113457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.183.1


Minutes Elapsed:  0.24557230869928995


<IPython.core.display.Javascript object>

In [35]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.823 (0.000)
MAE: 3.066 (0.000)
MAPE: 0.069 (0.000)
R2: 0.555 (0.000)


******
[TEST]
******
RMSE: 4.069 (0.000)
MAE: 3.441 (0.000)
MAPE: 0.081 (0.000)
R2: 0.520 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.300 (0.000)
MAE: 3.300 (0.000)
MAPE: 0.067 (0.000)
R2: 0.510 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.080 (0.000)
MAE: 3.510 (0.000)
MAPE: 0.091 (0.000)
R2: -1.085 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.880 (0.000)
MAE: 3.486 (0.000)
MAPE: 0.081 (0.000)
R2: 0.394 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 1

1. TIMESTEPS: 7

In [36]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM1, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.028809630870819


<IPython.core.display.Javascript object>

In [37]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.969 (0.000)
MAE: 2.408 (0.000)
MAPE: 0.053 (0.000)
R2: 0.732 (0.000)


******
[TEST]
******
RMSE: 3.792 (0.000)
MAE: 3.133 (0.000)
MAPE: 0.072 (0.000)
R2: 0.581 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.214 (0.000)
MAE: 3.510 (0.000)
MAPE: 0.073 (0.000)
R2: 0.501 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.188 (0.000)
MAE: 3.518 (0.000)
MAPE: 0.091 (0.000)
R2: -1.346 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.043 (0.000)
MAE: 2.534 (0.000)
MAPE: 0.056 (0.000)
R2: 0.622 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 1

1. TIMESTEPS: 14

In [38]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM1, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.6380537827809651


<IPython.core.display.Javascript object>

In [39]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.634 (0.000)
MAE: 3.028 (0.000)
MAPE: 0.068 (0.000)
R2: 0.597 (0.000)


******
[TEST]
******
RMSE: 4.285 (0.000)
MAE: 3.681 (0.000)
MAPE: 0.086 (0.000)
R2: 0.459 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 5.455 (0.000)
MAE: 4.576 (0.000)
MAPE: 0.097 (0.000)
R2: 0.170 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.734 (0.000)
MAE: 3.145 (0.000)
MAPE: 0.082 (0.000)
R2: -0.938 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.761 (0.000)
MAE: 3.508 (0.000)
MAPE: 0.082 (0.000)
R2: 0.417 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 2

1. TIMESTEPS: 1

In [40]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM2, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.38945345878601073


<IPython.core.display.Javascript object>

In [41]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.911 (0.000)
MAE: 3.126 (0.000)
MAPE: 0.069 (0.000)
R2: 0.535 (0.000)


******
[TEST]
******
RMSE: 3.926 (0.000)
MAE: 3.260 (0.000)
MAPE: 0.075 (0.000)
R2: 0.554 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.376 (0.000)
MAE: 3.289 (0.000)
MAPE: 0.067 (0.000)
R2: 0.492 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.312 (0.000)
MAE: 2.758 (0.000)
MAPE: 0.071 (0.000)
R2: -0.373 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.057 (0.000)
MAE: 3.671 (0.000)
MAPE: 0.083 (0.000)
R2: 0.337 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 2

1. TIMESTEPS: 7

In [42]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM2, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.3380911588668822


<IPython.core.display.Javascript object>

In [43]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.037 (0.000)
MAE: 2.440 (0.000)
MAPE: 0.054 (0.000)
R2: 0.719 (0.000)


******
[TEST]
******
RMSE: 3.625 (0.000)
MAE: 2.973 (0.000)
MAPE: 0.070 (0.000)
R2: 0.617 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.062 (0.000)
MAE: 3.290 (0.000)
MAPE: 0.069 (0.000)
R2: 0.536 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.169 (0.000)
MAE: 3.531 (0.000)
MAPE: 0.092 (0.000)
R2: -1.325 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 2.666 (0.000)
MAE: 2.268 (0.000)
MAPE: 0.052 (0.000)
R2: 0.710 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 2

1. TIMESTEPS: 14

In [44]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM2, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.7514315247535706


<IPython.core.display.Javascript object>

In [45]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.597 (0.000)
MAE: 2.958 (0.000)
MAPE: 0.068 (0.000)
R2: 0.605 (0.000)


******
[TEST]
******
RMSE: 4.186 (0.000)
MAE: 3.550 (0.000)
MAPE: 0.086 (0.000)
R2: 0.484 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 3.688 (0.000)
MAE: 2.962 (0.000)
MAPE: 0.062 (0.000)
R2: 0.621 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.903 (0.000)
MAE: 4.324 (0.000)
MAPE: 0.113 (0.000)
R2: -2.341 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.847 (0.000)
MAE: 3.311 (0.000)
MAPE: 0.080 (0.000)
R2: 0.390 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 3

1. TIMESTEPS: 1

In [46]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM3, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.44194891850153606


<IPython.core.display.Javascript object>

In [47]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.069 (0.000)
MAE: 3.285 (0.000)
MAPE: 0.072 (0.000)
R2: 0.496 (0.000)


******
[TEST]
******
RMSE: 4.037 (0.000)
MAE: 3.302 (0.000)
MAPE: 0.075 (0.000)
R2: 0.528 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.674 (0.000)
MAE: 3.496 (0.000)
MAPE: 0.070 (0.000)
R2: 0.421 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 2.982 (0.000)
MAE: 2.447 (0.000)
MAPE: 0.063 (0.000)
R2: -0.114 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.308 (0.000)
MAE: 3.894 (0.000)
MAPE: 0.088 (0.000)
R2: 0.252 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 3

1. TIMESTEPS: 7

In [48]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM3, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.4573658545811972


<IPython.core.display.Javascript object>

In [49]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.305 (0.000)
MAE: 2.663 (0.000)
MAPE: 0.057 (0.000)
R2: 0.668 (0.000)


******
[TEST]
******
RMSE: 3.618 (0.000)
MAE: 2.993 (0.000)
MAPE: 0.067 (0.000)
R2: 0.619 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.408 (0.000)
MAE: 3.644 (0.000)
MAPE: 0.073 (0.000)
R2: 0.454 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.209 (0.000)
MAE: 2.602 (0.000)
MAPE: 0.068 (0.000)
R2: -0.377 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.298 (0.000)
MAE: 2.855 (0.000)
MAPE: 0.063 (0.000)
R2: 0.556 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 3

1. TIMESTEPS: 14

In [50]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM3, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  3.7436386664708454


<IPython.core.display.Javascript object>

In [51]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.822 (0.000)
MAE: 3.150 (0.000)
MAPE: 0.072 (0.000)
R2: 0.555 (0.000)


******
[TEST]
******
RMSE: 5.148 (0.000)
MAE: 4.269 (0.000)
MAPE: 0.101 (0.000)
R2: 0.219 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 5.725 (0.000)
MAE: 4.600 (0.000)
MAPE: 0.094 (0.000)
R2: 0.086 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 5.715 (0.000)
MAE: 5.004 (0.000)
MAPE: 0.131 (0.000)
R2: -3.539 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.124 (0.000)
MAE: 3.426 (0.000)
MAPE: 0.082 (0.000)
R2: 0.299 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 4

1. TIMESTEPS: 1

In [52]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM4, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.3538273056348165


<IPython.core.display.Javascript object>

In [53]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.168 (0.000)
MAE: 3.430 (0.000)
MAPE: 0.076 (0.000)
R2: 0.472 (0.000)


******
[TEST]
******
RMSE: 4.213 (0.000)
MAE: 3.615 (0.000)
MAPE: 0.083 (0.000)
R2: 0.486 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.662 (0.000)
MAE: 3.618 (0.000)
MAPE: 0.074 (0.000)
R2: 0.424 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.341 (0.000)
MAE: 2.829 (0.000)
MAPE: 0.073 (0.000)
R2: -0.398 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.523 (0.000)
MAE: 4.289 (0.000)
MAPE: 0.098 (0.000)
R2: 0.176 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 4

1. TIMESTEPS: 7

In [54]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM4, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.657843554019928


<IPython.core.display.Javascript object>

In [55]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.097 (0.000)
MAE: 2.514 (0.000)
MAPE: 0.057 (0.000)
R2: 0.708 (0.000)


******
[TEST]
******
RMSE: 3.367 (0.000)
MAE: 2.858 (0.000)
MAPE: 0.068 (0.000)
R2: 0.670 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 3.058 (0.000)
MAE: 2.461 (0.000)
MAPE: 0.051 (0.000)
R2: 0.737 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.828 (0.000)
MAE: 3.265 (0.000)
MAPE: 0.085 (0.000)
R2: -0.961 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.154 (0.000)
MAE: 2.799 (0.000)
MAPE: 0.065 (0.000)
R2: 0.594 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 4

1. TIMESTEPS: 14

In [56]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM4, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.8221040487289428


<IPython.core.display.Javascript object>

In [57]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 30.733 (0.000)
MAE: 25.017 (0.000)
MAPE: 0.562 (0.000)
R2: -27.805 (0.000)


******
[TEST]
******
RMSE: 28.353 (0.000)
MAE: 22.396 (0.000)
MAPE: 0.510 (0.000)
R2: -22.696 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 36.694 (0.000)
MAE: 30.782 (0.000)
MAPE: 0.662 (0.000)
R2: -36.553 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 24.025 (0.000)
MAE: 16.905 (0.000)
MAPE: 0.424 (0.000)
R2: -79.230 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 24.829 (0.000)
MAE: 21.159 (0.000)
MAPE: 0.477 (0.000)
R2: -24.410 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 5

1. TIMESTEPS: 1

In [58]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM5, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.5891015450159709


<IPython.core.display.Javascript object>

In [59]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.888 (0.000)
MAE: 3.033 (0.000)
MAPE: 0.071 (0.000)
R2: 0.540 (0.000)


******
[TEST]
******
RMSE: 4.365 (0.000)
MAE: 3.572 (0.000)
MAPE: 0.086 (0.000)
R2: 0.448 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.054 (0.000)
MAE: 3.162 (0.000)
MAPE: 0.065 (0.000)
R2: 0.564 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 5.185 (0.000)
MAE: 4.628 (0.000)
MAPE: 0.120 (0.000)
R2: -2.366 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.776 (0.000)
MAE: 2.967 (0.000)
MAPE: 0.072 (0.000)
R2: 0.426 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 5

1. TIMESTEPS: 7

In [60]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM5, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.9220783631006877


<IPython.core.display.Javascript object>

In [61]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.786 (0.000)
MAE: 2.194 (0.000)
MAPE: 0.050 (0.000)
R2: 0.764 (0.000)


******
[TEST]
******
RMSE: 3.379 (0.000)
MAE: 2.653 (0.000)
MAPE: 0.062 (0.000)
R2: 0.668 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 3.703 (0.000)
MAE: 2.936 (0.000)
MAPE: 0.060 (0.000)
R2: 0.614 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.263 (0.000)
MAE: 3.613 (0.000)
MAPE: 0.094 (0.000)
R2: -1.432 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 2.003 (0.000)
MAE: 1.632 (0.000)
MAPE: 0.038 (0.000)
R2: 0.836 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 5

1. TIMESTEPS: 14

In [62]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM5, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  2.3256359537442526


<IPython.core.display.Javascript object>

In [63]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.956 (0.000)
MAE: 2.364 (0.000)
MAPE: 0.053 (0.000)
R2: 0.734 (0.000)


******
[TEST]
******
RMSE: 3.381 (0.000)
MAE: 2.737 (0.000)
MAPE: 0.064 (0.000)
R2: 0.663 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.330 (0.000)
MAE: 3.500 (0.000)
MAPE: 0.073 (0.000)
R2: 0.477 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.194 (0.000)
MAE: 2.604 (0.000)
MAPE: 0.068 (0.000)
R2: -0.418 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 2.714 (0.000)
MAE: 2.318 (0.000)
MAPE: 0.054 (0.000)
R2: 0.696 (0.000)




<IPython.core.display.Javascript object>

# Saving the results

In [64]:
path = f"../../../../../../../reports/results/global_models/inn/all_cements/pre_training/full/"
filename = f"BidirectionalLSTM_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [65]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [66]:
columns_to_standardize = ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]

<IPython.core.display.Javascript object>

In [67]:
ddf = pd.concat(results_to_save).reset_index(drop=True)
ddf_copy = ddf.copy()
ddf_copy = ddf_copy[ddf_copy["Plant"].eq("INN")].reset_index(drop=True)

scaler = StandardScaler()
dddf_copy = ddf_copy.copy()
dddf_copy[columns_to_standardize] = scaler.fit_transform(
    ddf_copy[columns_to_standardize]
).copy()

ddf_copy["SCPM"] = (
    dddf_copy[["RMSE Test", "MAE Test", "MAPE Test"]].sum(axis=1) - dddf_copy["R2 Test"]
)
ddf_copy[ddf_copy["SCPM"].eq(ddf_copy["SCPM"].min())]

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,Cross Validation Params,RMSE Train,MAE Train,MAPE Train,R2 Train,RMSE Test,MAE Test,MAPE Test,R2 Test,SCPM
13,Global Model,INN,INN,Chemical + Mineralogical,"(3180, 13)",7,Bi-LSTM14,None,Standard Scaler,None,...,"{""train_size"": 0.8, ""test_size"": 0.2}",2.785546,2.193544,0.049614,0.763846,3.378901,2.653115,0.062488,0.667525,-1.448464


<IPython.core.display.Javascript object>

In [79]:
ddf = pd.concat(results_to_save).reset_index(drop=True)
ddf_copy = ddf.copy()
ddf_copy = ddf_copy[ddf_copy["Plant"].eq("INN")]
scaler = StandardScaler()
dddf_copy = ddf_copy.copy()
dddf_copy[columns_to_standardize] = scaler.fit_transform(
    ddf_copy[columns_to_standardize]
).copy()

ddf_copy["SCPM"] = (
    dddf_copy[["RMSE Test", "MAE Test", "MAPE Test"]].sum(axis=1) - dddf_copy["R2 Test"]
)
min_row = ddf_copy[ddf_copy["SCPM"].eq(ddf_copy["SCPM"].min())]
ddf.merge(min_row[["Timesteps", "Model"]], on=("Timesteps", "Model"), how="inner")

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,Cross Validation,Cross Validation Params,RMSE Train,MAE Train,MAPE Train,R2 Train,RMSE Test,MAE Test,MAPE Test,R2 Test
0,Global Model,INN,INN,Chemical + Mineralogical,"(3180, 13)",7,Bi-LSTM14,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.785546,2.193544,0.049614,0.763846,3.378901,2.653115,0.062488,0.667525
1,Global Model,INN,partner_iv,Chemical + Mineralogical,"(3180, 13)",7,Bi-LSTM14,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.785546,2.193544,0.049614,0.763846,3.703023,2.936317,0.060155,0.614410
2,Global Model,INN,partner_ii,Chemical + Mineralogical,"(3180, 13)",7,Bi-LSTM14,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.785546,2.193544,0.049614,0.763846,4.263354,3.613454,0.093510,-1.431899
3,Global Model,INN,partner_i,Chemical + Mineralogical,"(3180, 13)",7,Bi-LSTM14,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.785546,2.193544,0.049614,0.763846,2.002947,1.632012,0.037799,0.836234


<IPython.core.display.Javascript object>